Exercise 8
===
Let's create an autoencoder network that pretrains on MNIST images.

In [1]:
%autosave 60

import tensorflow as tf
import numpy as np
from tensorflow.contrib.layers import batch_norm, dropout
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Autosaving every 60 seconds
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
import math
from sklearn.base import BaseEstimator, ClassifierMixin
import tensorflow as tf

def he_normal_initialisation(n_inputs, n_outputs):
    stddev = np.power(2 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.truncated_normal((n_inputs, n_outputs), stddev=stddev)

def he_uniform_initialisation(n_inputs, n_outputs):
    r = np.power(6 / (n_inputs + n_outputs), 1 / np.sqrt(2))
    # truncated normal distributions limit the size of the weights, speeding up the training time.
    return tf.random_uniform((n_inputs, n_outputs), -r, r)

def create_next_batch_fn(data, sequence_lengths, targets, batch_size):
    assert len(data) == len(sequence_lengths) and len(data) == len(targets)
    current_batch = 0
    def next_batch():
        nonlocal current_batch
        i = current_batch
        #print(current_batch)
        current_batch = (current_batch + batch_size) % len(data)
        return data[i:i+batch_size], sequence_lengths[i:i+batch_size], targets[i:i+batch_size]
    return next_batch

In [16]:
import math

# Constants
input_spatial_size = 28
input_channels = 1
batch_size = 200

# Hyperparameters
learning_rate = 0.01
n_hidden_neurons = 100
l2_reg = 0.0001

initializer = tf.contrib.layers.variance_scaling_initializer()
activation = tf.nn.relu
regularizer = tf.contrib.layers.l2_regularizer(l2_reg)

with tf.device("/gpu:0"):
    n_input_neurons = input_spatial_size ** 2
    X = tf.placeholder(tf.float32, shape=(None, n_input_neurons), name="input") 
    weights1 = initializer([n_input_neurons, n_hidden_neurons])
    biases1 = tf.Variable(tf.zeros(n_hidden_neurons))
    n_output_neurons = n_input_neurons
    weights2 = initializer([n_hidden_neurons, n_output_neurons])
    biases2 = tf.Variable(tf.zeros(n_output_neurons))
    
    hidden1 = activation(tf.matmul(X, weights1) + biases1)
    outputs = tf.matmul(hidden1, weights2) + biases2

    with tf.name_scope("loss"):
        reconstruction_loss = tf.reduce_mean(tf.square(outputs - X), name="loss")
        regularisation_loss = regularizer(weights1) + regularizer(weights2)
        loss = reconstruction_loss + regularisation_loss

    with tf.name_scope("training"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_autoencoder_model.ckpt"
early_stopping_checkpoint_path = "./checkpoints/mnist_autoencoder_model_early_stopping.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

loss_summary = tf.summary.scalar('loss', loss)
summary_op = tf.summary.merge([loss_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

epochs = 20
n_batches = int(np.ceil(len(mnist.train.images) // batch_size))

early_stopping_check_frequency = n_batches // 4
early_stopping_check_limit = n_batches * 2

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
session = sess
sess.run(init)
#saver.restore(sess, interim_checkpoint_path)

best_loss = 1000000000.0
best_loss_step = 0
for epoch in range(epochs):
    print("epoch", epoch)
    for batch_index in range(n_batches):
        step = epoch * n_batches + batch_index
        # TODO: replace this with code that gets a batch from X and y.
        X_batch, y_batch = mnist.train.next_batch(batch_size)
        if batch_index % 10 == 0:
            summary_str = summary_op.eval(session=sess, feed_dict={X: X_batch})
            file_writer.add_summary(summary_str, step)
        t, l = sess.run([training_op, loss], feed_dict={X: X_batch})
        if batch_index % 10 == 0: print("loss:", l)
        # Early stopping check
        if batch_index % early_stopping_check_frequency == 0:
            if l < best_loss:
                saver.save(sess, early_stopping_checkpoint_path)
                best_loss = l
                best_loss_step = step
            elif step >= (best_loss_step + early_stopping_check_limit):
                print("Stopping early during epoch", epoch)
                break
    else:
        continue
    break
    save_path = saver.save(sess, interim_checkpoint_path)
saver.restore(sess, early_stopping_checkpoint_path)
save_path = saver.save(sess, "./checkpoints/mnist_autoencoder_model_final.ckpt")

epoch 0
loss: 0.398486
loss: 0.406053
loss: 0.288601
loss: 0.253259
loss: 0.223293
loss: 0.217363
loss: 0.218384
loss: 0.20213
loss: 0.194036
loss: 0.189517
loss: 0.186512
loss: 0.190104
loss: 0.17687
loss: 0.171117
loss: 0.178589
loss: 0.175422
loss: 0.175482
loss: 0.17309
loss: 0.165574
loss: 0.17004
loss: 0.170812
loss: 0.166065
loss: 0.16314
loss: 0.165421
loss: 0.169418
loss: 0.167816
loss: 0.162997
loss: 0.165743
epoch 1
loss: 0.165058
loss: 0.168027
loss: 0.165225
loss: 0.161505
loss: 0.162054
loss: 0.161413
loss: 0.160095
loss: 0.163863
loss: 0.160572
loss: 0.157025
loss: 0.155734
loss: 0.160782
loss: 0.161647
loss: 0.159888
loss: 0.164607
loss: 0.15918
loss: 0.164338
loss: 0.164245
loss: 0.162116
loss: 0.158272
loss: 0.155733
loss: 0.161838
loss: 0.158578
loss: 0.159033
loss: 0.157575
loss: 0.16423
loss: 0.159034
loss: 0.158266
epoch 2
loss: 0.159144
loss: 0.157479
loss: 0.156775
loss: 0.158491
loss: 0.162329
loss: 0.158506
loss: 0.158058
loss: 0.157791
loss: 0.159833
loss: 0.